In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import re
import os

In [ ]:
def get_soup(url):
    req = requests.get(url)      #html태그와 content내용을 Response 객체로 반환
    html = req.text                 
    soup = BeautifulSoup(html, 'html.parser')
    
    return soup

#################################################################################
def get_df_data_on_site(soup):
    col = []
    data = []
    box = soup.select('#filing_table > tbody tr')   #n 개의 div
    title = soup.select('#filing_table > thead td')   #n 개의 div
    
    for txt in title:
        col.append(txt.text)
        
    for row in box:
        row_data = []
        boxbox = row.select('td')
        
        for txt in boxbox:
            row_data.append(txt.text)
            
        data.append(row_data)
        
    return col, data

#################################################################################
def refine_df(col, data):
    df = pd.DataFrame(data, columns=col)
    
    p_date = re.compile(r'.{10}')
    p_word = re.compile(r'[a-zA-Z]*$')
    
    drop_col = ['Filing']
    df.drop(drop_col, axis=1, inplace=True)
    
    df['Type'] = df['TransactionDate']

    for i in range(len(df)):
        m_date = p_date.search(df.iloc[i]['TransactionDate'])
        m_word = p_word.search(df.iloc[i]['TransactionDate'])
        Y, M, D, h, m = split_datetime(df['ReportedDateTime'].iloc[i])
        
        df['Type'].iloc[i] = m_word.group()
        df['TransactionDate'].iloc[i] = m_date.group()[2:]
        df['ReportedDateTime'].iloc[i] = '{}-{}-{} {}:{}'.format(Y, M, D, h, m)
        
        version = '.{}.{}.{}.{}.{}'.format(Y, M, D, h, m)
        
    return df, version

#################################################################################
def split_datetime(datetype):  # ex) '2021-04-2911:20 pm' / '2021-04-28:02 am'
    date = datetype[2:10]
    time = datetype[10:-3]
    ap = datetype[-2:]
    Y, M, D = date.split('-')
    h, m = time.split(':')

    if ap[0] == 'p': 
        h = str((int(h)+12)) if h!='0' else '00'
    else: 
        if len(h) == 1:
            h = '0'+h

    return Y, M, D, h, m

#################################################################################
def df_name(name):
    return locals()['df_{}'.format(name.lower())]

#################################################################################
def df_from_db(tic):
    p = re.compile(tic)
    dirname = 'D:/AI/pjt2/DB'
    filenames = os.listdir(dirname)
    for filename in filenames:
        if p.search(filename) : break
    
    return pd.read_excel('{}/{}'.format(dirname, filename))

#################################################################################
def version_from_db(tic):
    p = re.compile(tic)
    dirname = 'D:/AI/pjt2/DB'
    filenames = os.listdir(dirname)
    for filename in filenames:
        if p.search(filename) : break

    return filename[-19:-5].split('.')

#################################################################################
def version_from_web(df):
    temp = df.iloc[-1]['ReportedDateTime']
    temp = temp.split(' ')
    
    return temp[0].split('-')+temp[1].split(':')

#################################################################################
def update_new_data(web_df, tic):
    
    if version_from_db(tic) != version_from_web(web_df):
        db_df = df_from_db(tic)
        
        for datetime in web_df[::-1]['ReportedDateTime']:
            if db_df.iloc[-1]['ReportedDateTime'] == datetime:
                new_df = pd.concat(db_df, web_df.iloc[-(i+1):-1])
                break
        
    return new_df

# ndf = update_new_data(df,'apple')
# ndf

In [ ]:
df.iloc[-2:-1]

,TransactionDate,ReportedDateTime,Company,Symbol,InsiderRelationship,SharesTraded,AveragePrice,TotalAmount,SharesOwned,Type
10,21-04-01,21-04-05 18:36,Apple Inc.,AAPL,WILLIAMS JEFFREY ECOO,"133,867",$125.74,"$16,832,436","489,490(Direct)",Sale


In [ ]:
if __name__ == "__main__":    
    url_lib = {'TESLA':'1318605', 
               'APPLE':'320193', 
               'PALANTIR':'1321655', 
               'COUPANG':'1834584', 
               'UNITY':'1810806' }
    url_form = 'https://www.secform4.com/insider-trading/{}.htm'
    
    for name,form in url_lib.items():
        print('making dataframe.... {}....'.format(name))
        soup = get_soup(url_form.format(form))
        col, data = get_df_data_on_site(soup)
        df, version = refine_df(col, data)
    
        dir_DB = 'D:/AI/pjt2/DB'
        filename = 'df_{}{}.xlsx'.format(name.lower(), version)
        
        print('saving to DB as \'{}\''.format(filename))
        
        df.to_excel('{}/{}'.format(dir_DB, filename), index=False)
        
    print('done!!')

making dataframe.... TESLA....
saved to DB as 'df_tesla.21.04.29.20.20.xlsx'
making dataframe.... APPLE....
saved to DB as 'df_apple.21.04.19.18.31.xlsx'
making dataframe.... PALANTIR....
saved to DB as 'df_palantir.21.04.21.21.02.xlsx'
making dataframe.... COUPANG....
saved to DB as 'df_coupang.21.03.15.19.00.xlsx'
making dataframe.... UNITY....
saved to DB as 'df_unity.21.04.12.14.44.xlsx'
done!!


In [ ]:
######################################################################
################# 텔레그램에 구현해야 하는 내용 ######################
######################################################################

tic = input('티커를 입력하시오')

while not(tic.upper() in url_lib):
    print('해당 회사({})는 지원하지 않습니다.\n'.format(tic),list(url_lib.keys()),'\n해당 기업을 지원합니다.')
    tic = input('\n티커를 입력하시오 :')

name = tic.upper()
url_form = 'https://www.secform4.com/insider-trading/{}.htm'
print('get data from site.... {}....'.format(name), end='')
soup = get_soup(url_form.format(url_lib.get(name)))
print('success')
print('parsing site.... {}....'.format(name), end='')
col, data = get_df_data_on_site(soup)
print('success')
print('making dataframe.... {}....'.format(name), end='')
globals()['df_{}'.format(name.lower())] = make_refine_df(col, data)
print('success')
print('reframing dataframe.... {}....'.format(name), end='')
globals()['df_{}'.format(name.lower())] = split_using_regex(globals()['df_{}'.format(name.lower())])
print('success\n'+('=' * 50))
num = int(input('몇개의 거래내역을 불러오시겠습니까? :'))
# while not(num.isdigit()):
#     num = int(input('숫자를 입력하세요\n몇개의 거래내역을 불러오시겠습니까? :'))

for i in range(df_name(name).shape[0]-1,df_name(name).shape[0]-1-num,-1):
    df = df_name(name).iloc[i]
    print('\'{}\'{}s {} shares in price as {} on {} (reported {})\n'\
          .format(df['InsiderRelationship'],df['Type'],df['SharesTraded'],df['AveragePrice'],
                  df['TransactionDate'],df['ReportedDateTime']))
    print('#'*50)
    
    
##############################################################################

In [ ]:
update_new_data(df, 'apple')

['21', '04', '19', '18', '31']
['21', '04', '12', '14', '44']


In [ ]:
df = df_from_db('apple')
df

,TransactionDate,ReportedDateTime,Company,Symbol,InsiderRelationship,SharesTraded,AveragePrice,TotalAmount,SharesOwned,Type
0,20-04-15,20-04-17 18:31,Apple Inc.,AAPL,O'BRIEN DEIRDREVP,"9,137",$285.12,"$2,605,141","33,972(Direct)",Sale
1,20-05-08,20-05-12 18:30,Apple Inc.,AAPL,KONDO CHRISPrincipal Accounting Officer,"4,491",$305.62,"$1,372,539","7,370(Direct)",Sale
2,20-08-24,20-08-25 18:45,Apple Inc.,AAPL,COOK TIMOTHY DCEO,"265,160",$496.91,"$131,761,780","837,374(Direct)",Sale
3,20-10-01,20-10-05 18:33,Apple Inc.,AAPL,WILLIAMS JEFFREY ECOO,"257,343",$113.59,"$29,232,595","489,260(Direct)",Sale
4,20-10-09,20-10-13 18:30,Apple Inc.,AAPL,Maestri LucaCFO,"243,431",$116.89,"$28,454,649","110,272(Direct)",Sale
5,20-10-15,20-10-19 18:30,Apple Inc.,AAPL,KONDO CHRISPrincipal Accounting Officer,"14,840",$121.34,"$1,800,686","26,876(Direct)",Sale
6,20-10-15,20-10-19 18:32,Apple Inc.,AAPL,O'BRIEN DEIRDREVP,"31,200",$119.8,"$3,737,869","135,888(Direct)",Sale
7,20-11-03,20-11-05 18:30,Apple Inc.,AAPL,Adams Katherine L.VP,"17,000",$110.42,"$1,877,109","306,396(Direct)",Sale
8,21-02-01,21-02-03 18:30,Apple Inc.,AAPL,Adams Katherine L.VP,"17,000",$133.75,"$2,273,749","316,581(Direct)",Sale
9,21-02-01,21-02-03 18:36,Apple Inc.,AAPL,LEVINSON ARTHUR DDirector,"3,416",$135.6,"$463,210","4,532,724(IndirectDirect)",Sale
